In [1]:
import itertools
import operator
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_distances

In [2]:
!dir

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 2AA8-6EE5

 C:\Users\Sechang\1. DataScience\Project\tumblebug 디렉터리

2016-04-10  오후 08:54    <DIR>          .
2016-04-10  오후 08:54    <DIR>          ..
2016-04-10  오후 08:46    <DIR>          .ipynb_checkpoints
2016-04-03  오후 02:33            36,140 01. tumblebug_crawling_pratice.ipynb
2016-04-01  오후 10:13           165,735 01. tumblebug_project_crawling.ipynb
2016-04-09  오후 04:14            32,846 01. tumblebug_user_crawling_1.ipynb
2016-04-04  오전 11:40            32,936 01. tumblebug_user_crawling_2.ipynb
2016-04-04  오전 11:40            32,820 01. tumblebug_user_crawling_3.ipynb
2016-04-04  오전 11:40            33,761 01. tumblebug_user_crawling_4.ipynb
2016-04-04  오후 02:07            85,623 01. tumblebug_user_crawling_5-Copy1.ipynb
2016-04-04  오전 11:40            39,629 01. tumblebug_user_crawling_5.ipynb
2016-04-04  오전 12:33            56,644 02. tumblebug_grammar_level.ipynb
2016-04-01  오후 10:13             3,503 1. Crwadfunding crawling.ipynb
2016-

In [3]:
tb_df = pd.read_excel('tb_des_df.xlsx')

In [4]:
des_df = tb_df['description']

In [25]:
tb_users = pd.read_excel('tb_users_0410.xlsx')

In [6]:
des_df = pd.concat([tb_df['id'], tb_df.title, tb_df.category, des_df], axis=1)

In [7]:
des_df.head()

,id,title,category,description
0,12,사슴을 타고 온 아이,동화,\n 저를 모르시는 모든 분들께 인사드릴까요. 김가경이라고 합니다.영화와 ...
1,16,LETStudent,공간디자인,"\n안녕하세요 국민대학교 시각디자인학과 4학년에 재학중인 김문규, 민준기 입니다. ..."
2,21,Starting Up,다큐멘터리,\n 안녕하십니까 국민대학교 시각디자인과 졸업반에 정원도입니다. 이번에 제...
3,22,전방위이종문화리뷰지 나불나불,저널리즘,\nnabulnabul.com / twitter @nabulnabul\n1. 반가워...
4,23,개똥벌레: 프로그래밍 가능한 자전거 후미등,하드웨어,\n개똥벌레 소개개똥벌레는 소리로 빛의 세기를 조절하는 자전거 후미등 입니다. 자전...


In [8]:
import gensim

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
analyze = CountVectorizer(analyzer="word").build_analyzer()

In [11]:
sentences = []
for i in des_df['description']:
    words = analyze(i)
    sentences.append(words)

In [12]:
text_all=[]
for i in np.arange(len(sentences)):
    text = gensim.models.doc2vec.LabeledSentence(words = sentences[i], tags = ['doc_{number}'.format(number=i)])
    text_all.append(text)

In [13]:
model = gensim.models.Doc2Vec(alpha=0.025, min_alpha=0.025)  # use fixed learning rate
model.build_vocab(text_all)
for epoch in range(10):
    model.train(text_all)
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay
    print(epoch)

0
1
2
3
4
5
6
7
8
9


In [19]:
model.save('doc_vec.model')

In [15]:
# vector 담기
doc_vecs = []
for i in range(0, len(text_all)+1):
    vec = model.infer_vector('doc_{number}'.format(number=i))
    doc_vecs.append(vec)

# 유클리드 거리 구하기
dists = []
eu_dist = sp.spatial.distance.euclidean
for i, j in itertools.combinations(np.arange(len(doc_vecs)), 2):
    dist = eu_dist(doc_vecs[i], doc_vecs[j])
    dists.append((i, j, dist))

In [17]:
"""DataFrame에 넣기"""
eu_distance_df = pd.DataFrame(dists, columns=['doc_number_1', 'doc_number_2', 'euclidean_distance'])

In [21]:
model.save('doc_vec.model')
eu_distance_df.to_csv('doc_distance.csv')

In [18]:
sorted(eu_distance_df[eu_distance_df['doc_number_2'] == 1401]['euclidean_distance'], reverse=True)[:10]

[0.026160143315792084,
 0.026017207652330399,
 0.025855647400021553,
 0.025843935087323189,
 0.025780284777283669,
 0.025741815567016602,
 0.025671036913990974,
 0.025666665285825729,
 0.025648092851042747,
 0.025638898834586143]

In [22]:
sorted(eu_distance_df['euclidean_distance'], reverse=True)[:10]

[0.027584536001086235,
 0.027224572375416756,
 0.027201918885111809,
 0.027183499187231064,
 0.027164442464709282,
 0.027127301320433617,
 0.027120063081383705,
 0.027117909863591194,
 0.027112174779176712,
 0.02711009606719017]

In [23]:
sorted(eu_distance_df['euclidean_distance'], reverse=False)[:10]

[0.019510423764586449,
 0.019559178501367569,
 0.019568115472793579,
 0.019749943166971207,
 0.019751684740185738,
 0.019797086715698242,
 0.019839359447360039,
 0.01986008882522583,
 0.019878407940268517,
 0.01988535188138485]

In [39]:
user_grouped = tb_users.groupby('user')

In [40]:
user_grouped.get_group('/u/usertoneruyosa')

,user,id
71292,/u/usertoneruyosa,/project8-2
109066,/u/usertoneruyosa,/spff
110112,/u/usertoneruyosa,/lezhin_04_gongmyung2
113359,/u/usertoneruyosa,/beerguidebook
123141,/u/usertoneruyosa,/freedom_to_dream
130805,/u/usertoneruyosa,/singlelifeintokyo
145138,/u/usertoneruyosa,/dbd
151326,/u/usertoneruyosa,/peace


In [42]:
len(tb_users.user.value_counts())

109557

In [44]:
len(tb_users.user.unique())

109557

In [61]:
import time
start = time.clock()

a = pd.DataFrame()
for i in tb_users.user.unique():
    user = user_grouped.get_group(i)
    a = a.append(user)

end = time.clock()
print ("%.2gs" % (end-start))

9.1e+02s


In [53]:
len(tb_users)

169292

In [65]:
tb_user_gr = a.copy()

In [117]:
tb_user_gr.index = np.arange(len(tb_user_gr))

In [125]:
eu_distance_df.columns

Index(['doc_number_1', 'doc_number_2', 'euclidean_distance'], dtype='object')

In [134]:
sorted(eu_distance_df.loc[eu_distance_df['doc_number_1'] == 1121]['euclidean_distance'], reverse=True)[:10]

[0.026724549010396004,
 0.026436978951096535,
 0.02635437436401844,
 0.026195500046014786,
 0.026192314922809601,
 0.026188962161540985,
 0.02613234706223011,
 0.026128513738512993,
 0.026127943769097328,
 0.026085758581757545]

In [141]:
len(tb_df)

2802

In [190]:
def project_recommend(number):
    project = []
    titles = []
    categories = []
    if number <= 1401:
        dist = sorted(eu_distance_df.loc[eu_distance_df['doc_number_1'] == number]['euclidean_distance'], reverse=True)[:11];
        for i in dist:
            project_number = eu_distance_df.loc[eu_distance_df['euclidean_distance'] == i]['doc_number_2'];
            project.append(project_number);
    else:
        dist = sorted(eu_distance_df.loc[eu_distance_df['doc_number_2'] == number]['euclidean_distance'], reverse=True)[:11];
        for i in dist:
            project_number = eu_distance_df.loc[eu_distance_df['euclidean_distance'] == i]['doc_number_1'];
            project.append(project_number);
    for j in project:
        title = tb_df.ix[j]['title'];
        category = tb_df.ix[j]['category'];
        titles.append(title);
        categories.append(category)
    return ([titles], [categories])

In [198]:
project_recommend(2500)

([[1950    2015 신진연출가전 - 고독의 기원
   Name: title, dtype: object, 1971    심신기능활성 실천아카이브
   Name: title, dtype: object, 1614                     패션매거진「CUSZZ」창간호
   2028    한포먼스의 첫 프로젝트 공연 <나드리 : 나를 보여드리다>
   Name: title, dtype: object, 1209    해적X지도
   Name: title, dtype: object, 530    네거티브 칸나 27 - 2
   Name: title, dtype: object, 1142    드로잉 액션게임 '리틀스타'
   Name: title, dtype: object, 267        ME&YOU&EVERYONE WE HATE
   2274    연극 <외계인들 The Aliens> @국립극단
   Name: title, dtype: object, 444    성북동 일기
   Name: title, dtype: object, 600     제주도에서 록 페스티벌을! JET Fest
   2398          I.love.you.cookie
   Name: title, dtype: object, 2188    21세기 현자들을 위한 성性서, <춘화첩>
   Name: title, dtype: object, 1396         CIRCLINK - 새로운 모바일 리듬게임
   2415    독립운동가들을 기억하는 '27590:독립운동판결문'
   Name: title, dtype: object]], [[1950    연극
   Name: category, dtype: object, 1971    출판
   Name: category, dtype: object, 1614    출판
   2028    음악
   Name: category, dtype: object, 1209    보드게임
   Name: category, dtype: objec

In [199]:
d = sorted(eu_distance_df.loc[eu_distance_df['doc_number_1'] == 1121]['euclidean_distance'], reverse=True)[:10]

In [218]:
d[0]*10000

267.24549010396004

In [216]:
d[9]*10000

260.85758581757545